In [ ]:
!pip install geemap
import geemap

import ee
import os
import sys
import cv2
import time
import urllib
import imageio
import numpy as np
import pandas as pd
import matplotlib as mpl
from ee import batch
from matplotlib import cm
from google.colab import drive

ee.Authenticate()
ee.Initialize()

drive.mount('/content/gdrive/')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.7 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=d871a11570b096af13e5d2ae6c3fff83f74cf0f2455fc4126a70de

In [ ]:
siteName = 'Indian River Lagoon'
siteName_2char = 'IRL'
# aoi = ee.Geometry.Polygon(
        # [[[-81.25211704231316, 40.69010034533715],
        #   [-81.30979526496941, 40.57128714096595],
        #   [-81.27683628059441, 40.35605371044938],
        #   [-81.32902133918816, 40.23664763953814],
        #   [-81.58994663215691, 40.146429782371236],
        #   [-82.01017368293816, 40.083416034309785],
        #   [-82.19419467903191, 40.068704425699224],
        #   [-82.51005161262566, 39.93615696969347],
        #   [-82.55399692512566, 39.902453158253174],
        #   [-82.51005161262566, 39.860300069288385],
        #   [-81.71903598762566, 39.83710482583552],
        #   [-81.30155551887566, 39.86873276009167],
        #   [-81.12852085090691, 40.10442710451236],
        #   [-81.16971958137566, 40.276473107764815],
        #   [-81.07084262825066, 40.370703063923614],
        #   [-81.12302768684441, 40.512845346436144],
        #   [-81.15598667121941, 40.675520637717945],
        #   [-81.20267856575066, 40.69426539030674]]])

# upperLeft = [-80.763739, 28.621909]
# lowerRight = [-80.503601,  27.605387]
upperLeft = [-80.763739, 28.621909]
lowerRight = [-80.533771,   28.063011]
# Define a region of interest to filter our collection.
aoi = ee.Geometry.Rectangle(upperLeft[0], upperLeft[1], lowerRight[0], lowerRight[1])

In [ ]:
# aoi = (ee.Geometry.Polygon(
#         [[[-81.48841312904021, 39.955987994800424],
#           [-81.48841312904021, 39.859336443887486],
#           [-81.29340580482146, 39.859336443887486],
#           [-81.29340580482146, 39.955987994800424]]])).buffer(1000)

In [ ]:
# s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).filterBounds(aoi).filterDate('2022-01-01','2023-01-01')


In [ ]:
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
         .filterBounds(aoi)
         .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than', 20)
         .filterDate('2020-02-08','2020-02-09')
        #  .filter(ee.Filter.eq('system:index','20210705T160829_20210705T162039_T17TME').Not())
        #  .filter(ee.Filter.eq('system:index','20210730T160911_20210730T161940_T17TME').Not())
        #  .filter(ee.Filter.eq('system:index','20210804T160829_20210804T161533_T17TME').Not())
        #  .filter(ee.Filter.eq('system:index','20210809T160901_20210809T161432_T17TME').Not())
        #  .filter(ee.Filter.eq('system:index','20211112T161519_20211112T162021_T17TME').Not())

        )

In [ ]:
background = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(aoi)
              # .filter(ee.Filter.calendarRange(6,9,'month'))
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
              .map(lambda img: img.updateMask(img.select('QA60').eq(0)))
              .min())

map = geemap.Map()

map.addLayer(background,{'min':0,'max':2000})

map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# Define the Sentinel-2 bands to extract.
non_ee_band_list = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A']
native_res = 10
maxP = 1e9
wl = ee.Array([443.9,496.6,560.0,664.5,703.9,740.2,782.5,835.1,864.8])
# wl = ee.Array([442.7,492.7,559.8,664.4,704.1,740.5,782.8,832.8,864.7])

# Name the band combo for outputs.
bandCombo="B2-B8A"

""" Bands and Wavelength Array to select. You can add or subtract any you'd like here and the code
    below will account for the change throughout the workflow. """

# Use this for the derivative calculation.
band_list_early_length = len(non_ee_band_list)
# We use this everywhere else after the derivative calc (original band list x2).
band_list_length = band_list_early_length

# Use this in a couple for loops throughout the code.
band_list_forLoops = range(band_list_length)

In [ ]:
def derivative_images(water):

  """This section is devoted to modifying the image, making it suitable for
     the Derivative Spectra function, and running the DS function."""

  # This function masks the cloudy pixels and identifies the water pixels.
  def get_water(image):

    # Water with cloud check, but note this is the not bit addition method.
    water_pixels = (image.select('QA60').eq(0).And(image.select('SCL').eq(6)))

    # Mask all non-water pixels in the image.
    image = image.updateMask(water_pixels)

    return image

  # Run the LS8 mask on the test images.
  water_mask = get_water(water)

  # Binary reducer, which returns 1 everywhere that water hasn't been masked.
  mask_image = water_mask.reduce(ee.Reducer.anyNonZero())

  # Any pixels passing the pixel mask will now be polygonized.
  water_geom = mask_image.reduceToVectors(scale = native_res, geometry = aoi, maxPixels = maxP, bestEffort = True)

  # This function will select the bands of importance and clip the raster.
  def selAndClip(masked_image,geo):

    clipped = masked_image.select(non_ee_band_list).clip(geo)

    return clipped.toDouble()

  # Run the select and clip function on the test images, converts the pixels values to double precision.
  clipped_image = selAndClip(water_mask,water_geom)

  # Add code below to divide the reflectance bands by 10^4 to scale from 0 to 1
  clipped_image = clipped_image.multiply(0.0001)

  def derivativeSpectra(img,numBands):

    drdlraster = None

    for i in range(numBands):

      # Special case of the equation is required for the first band.
      if i == 0:
        drdlraster = img.expression(
          '(secondBand - firstBand)/(secondIndex - firstIndex)',
          {
          'secondBand': img.select(i+1),
          'firstBand': img.select(i),
          'secondIndex': wl.get([i+1]),
          'firstIndex': wl.get([i])
          }).rename('image_Index' + str(i) + '_derivative')

      # Special case of the equation is required for the last band.
      elif i == (numBands-1):

        drdlraster = img.expression(
          '(secondBand - firstBand)/(secondIndex - firstIndex)',
          {
          'secondBand': img.select(i),
          'firstBand': img.select(i-1),
          'secondIndex': wl.get([i]),
          'firstIndex': wl.get([i-1])
          }).rename('image_Index' + str(i) + '_derivative')

      # Apply equation for all other bands in the image.
      else:
        drdlraster = img.expression(
          '(secondBand - firstBand)/(secondIndex - firstIndex)',
          {
          'secondBand': img.select(i+1),
          'firstBand': img.select(i-1),
          'secondIndex': wl.get([i+1]),
          'firstIndex': wl.get([i-1])
          }).rename('image_Index' + str(i) + '_derivative')

      # Adds derivative bands to the input image.
      img = img.addBands(drdlraster)

    # JDO Modified this line to return both Ref and drdl bands, rather than return only the derivative bands, cast to a new image.
    return img.select([9,10,11,12,13,14,15,16,17])

  # Run the Derivative Spectra function on the test image.
  d_image = derivativeSpectra(clipped_image,band_list_early_length)

  return d_image

In [ ]:
derivative_list = s2.map(derivative_images)

# Run VPCA for each image

In [ ]:
corr3 = pd.read_csv('/content/gdrive/My Drive/test_FL_DEP/S2_avgCorrelationMatrix.csv').drop('Unnamed: 0',axis=1)
# corr3 = pd.read_csv('/content/gdrive/My Drive/S2_avgCorrelationMatrix.csv').drop('Unnamed: 0',axis=1)

In [ ]:
"""Section devoted to performing the Varimax Rotation."""

# This function is used to restructure the array after each pass through the rotation loop.
def vpca_toArray(initArr, a, b, xRot, yRot):
# Values of "i" and "j" are checked to return a new, reformatted version of the input array.
  if a == 0 and b == 1:
    initArr = ee.Array.cat([xRot,yRot,initArr.slice(1,2,3)],1)
  elif a == 0 and b == 2:
    initArr = ee.Array.cat([xRot,initArr.slice(1,1,2),yRot],1)
  elif a == 1 and b == 2:
    initArr = ee.Array.cat([initArr.slice(1,0,1),xRot,yRot],1)
  else:
    initArr = initArr
  return initArr

def varimax_rotation(input, band_list_length, flag = False):

  epsilon = ee.Number(2 * 0.00001)

  # Sqrt of communality.
  # (Communality: the proportion of variance that each item has in common with other items.)
  h = ((input.pow(2)).reduce(reducer=ee.Reducer.sum(),axes=([1]))).sqrt()
  # Extend "h" to match the dimensions of the input array.
  h_b = h.repeat(1,3)
  # Input is normalized by the sqrt of communality.
  bh = input.divide(h_b)

  """======================================================="""
  """Create list to be used for iteration. If you need to change
    the amount of iterations through the rotate() function,
    this is where you do it."""
  # Max seq. on which this step has been tested and worked: (0,6)
  seq = ee.List.sequence(0,6)
  """======================================================="""
  def rotate(current,prev):

    bh = ee.Array(prev)

    for i in range(2):
      for j in range(3):
        #  Select i'th column from the matrix.
        xx = bh.slice(1,i,i+1)
        #  Select the j'th column from the matrix.
        yy = bh.slice(1,j,j+1)

        #  Calculations below are made to determine phi.
        uu = (xx.pow(2)).subtract(yy.pow(2))

        vv = xx.multiply(yy).multiply(2)

        aa = uu.reduce(reducer=ee.Reducer.sum(),axes=([0])).get([0,0])

        bb = vv.reduce(reducer=ee.Reducer.sum(),axes=([0])).get([0,0])

        cc = (uu.pow(2)).subtract(vv.pow(2)).reduce(
                reducer=ee.Reducer.sum(),
                axes = ([0])
                ).get([0,0])

        dd = (uu.multiply(vv).multiply(2)).reduce(
                reducer = ee.Reducer.sum(),
                axes = ([0])
                ).get([0,0])

      num = dd.subtract((aa.multiply(bb).multiply(2)).divide(band_list_length))

      den = cc.subtract(((aa.pow(2)).subtract(bb.pow(2))).divide(band_list_length))

      phi = (den.atan2(num)).divide((band_list_length) - 1)

      # Determine if phi is greater than or equal to epsilon.
      evaluation = (phi.abs()).gte(epsilon)

      # Rotation takes place in server-side if statement.
      rot = ee.Algorithms.If(
                  condition=ee.Number(evaluation).eq(1),
                  trueCase=
                  {
                    'xx_rot': (xx.multiply(phi.cos())).add(yy.multiply((phi.sin()))),
                    'yy_rot': (xx.multiply((phi.multiply(-1)).sin())).add(yy.multiply(phi.cos()))
                  },
                  falseCase=None)
      # Dictionary contains the two rotations.
      rot = ee.Dictionary(rot)

      # Set the new value of "xx".
      setX = ee.Algorithms.If(
              condition=ee.Number(evaluation).eq(1),
              trueCase=ee.Dictionary(rot).getArray('xx_rot'),
              falseCase=xx)
      # Set the new value of "yy".
      setY = ee.Algorithms.If(
              condition=ee.Number(evaluation).eq(1),
              trueCase=ee.Dictionary(rot).getArray('yy_rot'),
              falseCase=yy)

      # Cast "xx" and "yy" into Array objects for the reformation function.
      xx = ee.Array(setX)
      yy = ee.Array(setY)

      # Reform the rotated array.
      bh = vpca_toArray(bh,i,j,xx,yy)

    # Checks the remaining number of pairs, determine if array has converged.
    return bh

  # Iterate through each position of the list defined above.
  seq = seq.iterate(rotate,bh)

  # Cast the result of the rotation as new bh.
  bh = ee.Array(seq)

  # Denormalize bh by multiplying it by the sqrt of communality.
  result = bh.multiply(h_b)

  fracVar = ((result.pow(2)).reduce(ee.Reducer.sum(),([0]))).divide(band_list_length)

  return ee.Dictionary({'result':result,'h':h,'fracVar':fracVar})

# Project original data onto component axis to create component scores.
def create_scores(data):

  # Calculate factor score coefficients.
  trans = data.transpose()
  v = trans.matrixMultiply(data)
  inverse = v.matrixInverse()
  b = data.matrixMultiply(inverse)

  return b

"""EE table export requires a Feature Collection.
   This step reformats the VPCA Loadings into a
   Feature, which can be accepted as an input in
   a Feature Collection."""

# Compile stats into EE Feature (necessary for export).
def createFeats_loads(sliver):
  row1 = sliver.get([0,0])
  row2 = sliver.get([0,1])
  row3 = sliver.get([0,2])
  row4 = sliver.get([0,3])
  row5 = sliver.get([0,4])
  row6 = sliver.get([0,5])
  row7 = sliver.get([0,6])
  row8 = sliver.get([0,7])
  row9 = sliver.get([0,8])
  row10 = sliver.get([0,9])

  rowsForCsv = {
      'Wavelengths': row1,
      'component 1': row2,
      'component 2': row3,
      'component 3': row4,
      'component 4': row5,
      'component 5': row6,
      'component 6': row7,
      'Comm_1-3': row8,
      'Comm_4-6': row9,
      'Comm_tot': row10
      }

  return ee.Feature(None,rowsForCsv)

def createFeats_eigens(sliver, title = None):
  row1 = sliver.get(0)
  row2 = sliver.get(1)
  row3 = sliver.get(2)
  row4 = sliver.get(3)
  row5 = sliver.get(4)
  row6 = sliver.get(5)

  rowsForCsv = {
      'Wavelengths': title,
      'component 1': row1,
      'component 2': row2,
      'component 3': row3,
      'component 4': row4,
      'component 5': row5,
      'component 6': row6,
      'Comm_1-3': 'Total',
      'Comm_4-6': sliver.slice(0,6).reduce(ee.Reducer.sum()),
      'Comm_tot': None
      }

  return ee.Feature(None, rowsForCsv)

def createFeats_fracVar(sliver, title = None):
  row1 = sliver.get([0])
  row2 = sliver.get([1])
  row3 = sliver.get([2])
  row4 = sliver.get([3])
  row5 = sliver.get([4])
  row6 = sliver.get([5])

  rowsForCsv = {
      'Wavelengths': title,
      'component 1': row1,
      'component 2': row2,
      'component 3': row3,
      'component 4': row4,
      'component 5': row5,
      'component 6': row6,
      'Comm_1-3': 'Total',
      'Comm_4-6': sliver.reduce(ee.Reducer.sum(), axes = [0]).get([0]),
      'Comm_tot': None
      }

  return ee.Feature(None, rowsForCsv)

In [ ]:
def fullAnalysis(image):
  # Next, convert our dataframe to a numpy array.
  corr_ls = corr3.to_numpy()
  # Eigenvalues and eigenvectors calculated using Numpy.
  eigens = np.linalg.eigh(corr_ls)
  # Extract the eigenvalues from eigens so that we can reverse the sort
  eigenVals = eigens[0]
  # Reverse the eigenvalue sort so that they are in descending order
  eigenVals = eigenVals[::-1]
  # Convert to an EE list.
  eigenVals = ee.List(list(eigenVals))

  # Build the eigenvector matrix

  # Rotate the eigenvector matrix 180 to place it in the correct orientiation to match
  # the eigenvalues sorted in descending order
  rotating = np.rot90(eigens[1],2)

  # Take each row of our eigenvector matrix and concatenate the rows in a new EE array.
  eigenVecs = ee.Array.cat([ee.Array(list(rotating[i])) for i in range(len(rotating))])

  # Reformat the array. May need to be multiplied by -1.
  eigenVecs = eigenVecs.reshape([band_list_length,band_list_length]).transpose()

  # ================= #
  """The following section is devoted to pre-processing
   the data for the Varimax Rotation."""

  # Determine whether or not each eigenvalue is positive.
  def checkEigs(eig):
      # Cast to an ee Number.
      eig = ee.Number(eig)
      check = ee.Algorithms.If(eig.lt(0),eig.multiply(-1),eig.multiply(1))
      return check

  # All eigenvalues should now be positive.
  eigenVals = eigenVals.map(checkEigs)

  # Determine square root of all eigenvalues to obtain the singular values.
  singvalue = ee.Array(eigenVals).sqrt()

  def formatData(rows,value):
    new_list = ee.List([])

    # Slice array and multiply by each row's respective positive eigenvalue.
    for i in band_list_forLoops:
      sliver = ee.Algorithms.If(ee.Number(i).eq(band_list_length - 1), rows.slice(0,i).multiply(value.get([i])),
                                                    rows.slice(0,i,i+1).multiply(value.get([i])))
      # Push formatted rows into new empty list.
      new_list = new_list.add(sliver)
    return new_list

  # Get our formatted information.
  pcaload = formatData(eigenVecs,singvalue)

  # Select the top three principal components for the rotation.
  pc1 = pcaload.getArray(0).project([1]).toList()
  pc2 = pcaload.getArray(1).project([1]).toList()
  pc3 = pcaload.getArray(2).project([1]).toList()
  # Second batch of PCs.
  pc4 = pcaload.getArray(3).project([1]).toList()
  pc5 = pcaload.getArray(4).project([1]).toList()
  pc6 = pcaload.getArray(5).project([1]).toList()


  # Create the input arrays for rotation.
  varimaxInput_1 = ee.Array([pc1,pc2,pc3]).transpose()
  varimaxInput_2 = ee.Array([pc4,pc5,pc6]).transpose()

  # THIS IS WHERE I USED TO HAVE THE VPCA FUNCTION.

  # Run the varimax on PCs 1-3, then PCs 4-6.
  vpca_set1 = varimax_rotation(varimaxInput_1, band_list_length)
  vpca_set2 = varimax_rotation(varimaxInput_2, band_list_length)

  # Retrieves the resultant array from the dictionary output.
  batch1 = ee.Array(vpca_set1.get('result'))
  batch2 = ee.Array(vpca_set2.get('result'))

  # Project original data onto component axis to create component scores.
  batch1_scores = create_scores(batch1)
  batch2_scores = create_scores(batch2)
  all_scores = ee.Array.cat([batch1_scores,batch2_scores],1)

  # Cast "b" into an Image object. This will be used for the matrix multiplication.
  b_img = ee.Image(all_scores)
  # Convert the derivative Image to an Array Image.
  derivative_array = ee.Image(image).toArray(0).toArray(1)
  # Transpose the Array Image.
  derivative_array = derivative_array.arrayTranspose()
  # Dot product of Array Image and "b" Image.
  applied_b = derivative_array.matrixMultiply(b_img)
  # Transpose the array. This is an optional step to project the bands onto a column.
  applied_b = applied_b.arrayTranspose()
  # newImage = ee.Image(applied_b)

  # Get "h" for the stats output, raise to 2nd power.
  communality_1 = ee.Array(vpca_set1.get('h')).pow(2)
  communality_2 = ee.Array(vpca_set2.get('h')).pow(2)

  # Get fractions of variance.
  frac_var_set1 = ee.Array(vpca_set1.get('fracVar'))
  frac_var_set2 = ee.Array(vpca_set2.get('fracVar'))
  joined_frac_var = ee.Array.cat([frac_var_set1,frac_var_set2],1)

  # ===== #
  """This code block sorts the principal components using the fraction
     of variance array."""

  # Combine the two sets of vpca results.
  combo = ee.Array.cat([batch1,batch2],1)
  # Append the fractions of variance to the bottom of the stack.
  useToSort = ee.Array.cat([combo, joined_frac_var],0)

  # Use the fractions of variance to sort the vpca results in ascending order.
  bottomVals = useToSort.slice(0,band_list_length)
  sorted = useToSort.sort(bottomVals)

  # Flip our components to descending order.
  finalComponents = (ee.Array.cat(
    arrays = [
              sorted.slice(1,5),
              sorted.slice(1,4,5),
              sorted.slice(1,3,4),
              sorted.slice(1,2,3),
              sorted.slice(1,1,2),
              sorted.slice(1,0,1)
              ],
    axis = 1))
  # ===== #

  # Project fractions of variance down to a one-dimensional array.
  frac_var_flat = finalComponents.slice(0,band_list_length).project([1])

  # Step 1: compute the sum of the frac_var array.
  frac_var_flat_total = frac_var_flat.reduce(ee.Reducer.sum(),[0])

  # Step 2: extract the single member of the "frac_var_flat_total" array.
  frac_var_flat_total = ee.Number(frac_var_flat_total.get([0]))

  # Step 3: divide original frac_var_flat by our total observed variance.
  ex_frac_var_flat = frac_var_flat.divide(frac_var_flat_total)

  # Step 4: compute the sum of the extracted frac_var array. Should be close to 1.0.
  ex_frac_var_flat_tot = ex_frac_var_flat.reduce(ee.Reducer.sum(),[0])

  # Get PC bands, save fraction of full image variance score to each. This is needed to sort the bands.
  one = ee.Image(applied_b.arrayProject([0]).arrayGet(0)).set('frac_score',frac_var_flat.get([0]))
  two = ee.Image(applied_b.arrayProject([0]).arrayGet(1)).set('frac_score',frac_var_flat.get([1]))
  three = ee.Image(applied_b.arrayProject([0]).arrayGet(2)).set('frac_score',frac_var_flat.get([2]))
  four = ee.Image(applied_b.arrayProject([0]).arrayGet(3)).set('frac_score',frac_var_flat.get([3]))
  five = ee.Image(applied_b.arrayProject([0]).arrayGet(4)).set('frac_score',frac_var_flat.get([4]))
  six = ee.Image(applied_b.arrayProject([0]).arrayGet(5)).set('frac_score',frac_var_flat.get([5]))

  # Push PC bands into image collection, sort by frac_score in descending order, flatten into a single image and rename the bands.
  finalImage = ee.ImageCollection([one,two,three,four,five,six]).sort('frac_score', False).toBands().rename(['PC_1','PC_2','PC_3','PC_4','PC_5','PC_6'])

  # Step 5: save the extracted fractions of variance, along with their total, to the image itself.
  finalImage = finalImage.set({
    'extracted_fractions_of_variance': ex_frac_var_flat,
    'total_variance': ex_frac_var_flat_tot.get([0]),
  })

  return finalImage

In [ ]:
pcs = derivative_list.map(fullAnalysis)

In [ ]:
max_min = (pcs.mean().reduceRegion(
    # reducer = ee.Reducer.percentile([2,98]),
    reducer = ee.Reducer.minMax(),
    geometry = aoi,
    scale = native_res,
    maxPixels = 1e5,
    bestEffort = True))

In [ ]:
max_min = max_min.getInfo()

#Optional

In [ ]:
print(max_min)

{'PC_1_max': 0.00018245249897160256, 'PC_1_min': -0.0003535690417263656, 'PC_2_max': 0.0019502452201231846, 'PC_2_min': -0.0006941860854033137, 'PC_3_max': 0.0013090323838677426, 'PC_3_min': -0.0010128539381540102, 'PC_4_max': 0.0020604913570656876, 'PC_4_min': -0.002809844997412524, 'PC_5_max': 0.0015312025742682134, 'PC_5_min': -0.000976021985034391, 'PC_6_max': 0.003152910655474265, 'PC_6_min': -0.0019456038698743628}


In [ ]:
minimum = min(max_min.values())
maximum = max(max_min.values())

# GIF Creation

In [ ]:
help(geemap.add_text_to_gif)

Help on function add_text_to_gif in module geemap.timelapse:

add_text_to_gif(in_gif, out_gif, xy=None, text_sequence=None, font_type='arial.ttf', font_size=20, font_color='#000000', add_progress_bar=True, progress_bar_color='white', progress_bar_height=5, duration=100, loop=0)
    Adds animated text to a GIF image.
    
    Args:
        in_gif (str): The file path to the input GIF image.
        out_gif (str): The file path to the output GIF image.
        xy (tuple, optional): Top left corner of the text. It can be formatted like this: (10, 10) or ('15%', '25%'). Defaults to None.
        text_sequence (int, str, list, optional): Text to be drawn. It can be an integer number, a string, or a list of strings. Defaults to None.
        font_type (str, optional): Font type. Defaults to "arial.ttf".
        font_size (int, optional): Font size. Defaults to 20.
        font_color (str, optional): Font color. It can be a string (e.g., 'red'), rgb tuple (e.g., (255, 127, 0)), or hex code (e

In [ ]:
def hex_colormap(palette_name = None):
  # "palette_name" must be a string.
  ramp = cm.get_cmap(palette_name)
  # Converts color values to hexadecimal format so we can use them.
  ramp_ls = [mpl.colors.rgb2hex(ramp(color)[:3]) for color in range(ramp.N)]

  return ramp_ls

spectral = hex_colormap('Spectral')
spectralr = hex_colormap('Spectral_r')

In [ ]:
# Use reduceColumns to get back our image ID info.
ids = pcs.reduceColumns(ee.Reducer.toList(), ['system:index']).getInfo()

In [ ]:
ids

{'list': ['20200208T160419_20200208T160420_T17RNM']}

In [ ]:
background_viz = (background.visualize(
  bands = ['B4','B3','B2'],
  min = 0,
  max = 2000))

In [ ]:
pc_1 = (pcs.map(
  lambda img: (ee.ImageCollection([background_viz,
              img.select('PC_1').visualize(min = max_min['PC_1_min']*0.5, max = max_min['PC_1_max']*0.5, palette = spectral)])
              .mosaic())))

pc_2 = (pcs.map(
  lambda img: (ee.ImageCollection([background_viz,
              img.select('PC_2').visualize(min = max_min['PC_2_min']*.20, max = max_min['PC_2_max']*.20, palette = spectralr)])
              .mosaic())))

pc_3 = (pcs.map(
  lambda img: (ee.ImageCollection([background_viz,
              img.select('PC_3').visualize(min = max_min['PC_3_min']*.25, max = max_min['PC_3_max']*.25, palette = spectralr)])
              .mosaic())))

pc_4 = (pcs.map(
  lambda img: (ee.ImageCollection([background_viz,
              img.select('PC_4').visualize(min = max_min['PC_4_min']*.10, max = max_min['PC_4_max']*.10, palette = spectral)])
              .mosaic())))

pc_5 = (pcs.map(
  lambda img: (ee.ImageCollection([background_viz,
              img.select('PC_5').visualize(min = max_min['PC_5_min']*.20, max = max_min['PC_5_max']*.20, palette = spectralr)])
              .mosaic())))

pc_6 = (pcs.map(
  lambda img: (ee.ImageCollection([background_viz,
              img.select('PC_6').visualize(min = max_min['PC_6_min']*.25, max = max_min['PC_6_max']*.25, palette = spectralr)])
              .mosaic())))

In [ ]:
gif1 = pc_1.getVideoThumbURL({'dimensions':1024, 'region':aoi, 'framesPerSecond':2})
gif2 = pc_2.getVideoThumbURL({'dimensions':1024, 'region':aoi, 'framesPerSecond':2})
gif3 = pc_3.getVideoThumbURL({'dimensions':1024, 'region':aoi, 'framesPerSecond':2})
gif4 = pc_4.getVideoThumbURL({'dimensions':1024, 'region':aoi, 'framesPerSecond':2})
gif5 = pc_5.getVideoThumbURL({'dimensions':1024, 'region':aoi, 'framesPerSecond':2})
gif6 = pc_6.getVideoThumbURL({'dimensions':1024, 'region':aoi, 'framesPerSecond':2})

gif_ls = [gif1,gif2,gif3,gif4,gif5,gif6]

In [ ]:
task = (batch.Export.image.toDrive(
  image = pc_1,
  description = 'pc1_finalProduct',
  dimensions = 720,
  region = aoi
))
task.start()

In [ ]:
task = (batch.Export.video.toDrive(
  collection = pc_1,
  description = 'pc1_finalProduct',
  dimensions = 720,
  framesPerSecond = 1,
  region = aoi
))

task.start()
task = (batch.Export.video.toDrive(
  collection = pc_2,
  description = 'pc2_finalProduct',
  dimensions = 720,
  framesPerSecond = 1,
  region = aoi
))

task.start()
task = (batch.Export.video.toDrive(
  collection = pc_3,
  description = 'pc3_finalProduct',
  dimensions = 720,
  framesPerSecond = 1,
  region = aoi
))

task.start()
task = (batch.Export.video.toDrive(
  collection = pc_4,
  description = 'pc4_finalProduct',
  dimensions = 720,
  framesPerSecond = 1,
  region = aoi
))

task.start()
task = (batch.Export.video.toDrive(
  collection = pc_5,
  description = 'pc5_finalProduct',
  dimensions = 720,
  framesPerSecond = 1,
  region = aoi
))

task.start()
task = (batch.Export.video.toDrive(
  collection = pc_6,
  description = 'pc6_finalProduct',
  dimensions = 720,
  framesPerSecond = 1,
  region = aoi
))

task.start()

Solution to GIF export found here: https://gis.stackexchange.com/questions/358838/export-image-collection-animation-generated-by-google-earth-engine

In [ ]:
i=1
for gif in gif_ls:
  gif_name = f'gdrive/My Drive/pc_forPresentation_pc{i}.gif'
  urllib.request.urlretrieve(gif, gif_name)
  i+=1

In [ ]:
# Get the names of all of our GIFs.
vid_names = [f"/content/gdrive/My Drive/pc_forPresentation_pc{i+1}.gif" for i in range(6)]

In [ ]:
vid_names

['/content/gdrive/My Drive/pc_forPresentation_pc1.gif',
 '/content/gdrive/My Drive/pc_forPresentation_pc2.gif',
 '/content/gdrive/My Drive/pc_forPresentation_pc3.gif',
 '/content/gdrive/My Drive/pc_forPresentation_pc4.gif',
 '/content/gdrive/My Drive/pc_forPresentation_pc5.gif',
 '/content/gdrive/My Drive/pc_forPresentation_pc6.gif']

In [ ]:
# Only if it's already a GIF.

for gif in vid_names:
  (geemap.add_text_to_gif(
      gif,
      f"{gif.replace('.gif','')}_textAdded.gif",
      xy=('2%', '2%'),
      text_sequence=[f"{ids['list'][name]} PC #{vid_names.index(gif)+1}" for name in range(len(ids['list']))],
      font_size=14,
      font_color='#ffffff',
      duration=1000,
      add_progress_bar=False))

In [ ]:
geemap.show_image('/content/gdrive/My Drive/pc_forPresentation_pc1_textAdded.gif', width=None, height=None)